In [2]:
import onmt
from onmt.inputters.inputter import _load_vocab, _build_fields_vocab, get_fields, IterOnDevice
from onmt.inputters.corpus import ParallelCorpus
from onmt.inputters.dynamic_iterator import DynamicDatasetIter
from onmt.translate import GNMTGlobalScorer, Translator, TranslationBuilder
from onmt.utils.misc import set_random_seed

In [7]:
import yaml
import torch
import torch.nn as nn
from argparse import Namespace
from collections import defaultdict, Counter

In [4]:
# enable logging
from onmt.utils.logging import init_logger, logger
init_logger()

<RootLogger root (INFO)>

In [5]:
is_cuda = torch.cuda.is_available()
set_random_seed(1111, is_cuda)

Dummy dataset is downloaded with this: `wget https://s3.amazonaws.com/opennmt-trainingdata/toy-ende.tar.gz`

## Build vocabs

In [8]:
yaml_config = """
## Where the samples will be written
save_data: toy-ende/run/example
## Where the vocab(s) will be written
src_vocab: toy-ende/run/example.vocab.src
tgt_vocab: toy-ende/run/example.vocab.tgt
# Corpus opts:
data:
    corpus:
        path_src: toy-ende/src-train.txt
        path_tgt: toy-ende/tgt-train.txt
        transforms: []
        weight: 1
    valid:
        path_src: toy-ende/src-val.txt
        path_tgt: toy-ende/tgt-val.txt
        transforms: []
"""
config = yaml.safe_load(yaml_config)
with open("toy-ende/config.yaml", "w") as f:
    f.write(yaml_config)

Create the `config.yaml` file that used for building vocab

In [10]:
from onmt.utils.parse import ArgumentParser
parser = ArgumentParser(description='build_vocab.py')

Create the arguments parser for execution with `build_vocab.py` file
 
**TO DO:** Detailize the vocabulary building process used for this translation task: english to french

In [11]:
from onmt.opts import dynamic_prepare_opts
dynamic_prepare_opts(parser, build_vocab_only=True)

In [12]:
base_args = (["-config", "toy-ende/config.yaml", "-n_sample", "10000"])
opts, unknown = parser.parse_known_args(base_args)

The above two cells creating a ``wrapper'' for the arguments to be parsed for building vocab

In [13]:
opts

Namespace(config='toy-ende/config.yaml', data="{'corpus': {'path_src': 'toy-ende/src-train.txt', 'path_tgt': 'toy-ende/tgt-train.txt', 'transforms': [], 'weight': 1}, 'valid': {'path_src': 'toy-ende/src-val.txt', 'path_tgt': 'toy-ende/tgt-val.txt', 'transforms': []}}", dump_samples=False, insert_ratio=0.0, mask_length='subword', mask_ratio=0.0, n_sample=10000, num_threads=1, overwrite=False, permute_sent_ratio=0.0, poisson_lambda=3.0, random_ratio=0.0, replace_length=-1, reversible_tokenization='joiner', rotate_ratio=0.0, save_config=None, save_data='toy-ende/run/example', seed=-1, share_vocab=False, skip_empty_level='warning', src_feats_vocab=None, src_onmttok_kwargs="{'mode': 'none'}", src_seq_length=200, src_subword_alpha=0, src_subword_model=None, src_subword_nbest=1, src_subword_type='none', src_subword_vocab='', src_vocab='toy-ende/run/example.vocab.src', src_vocab_threshold=0, switchout_temperature=1.0, tgt_onmttok_kwargs="{'mode': 'none'}", tgt_seq_length=200, tgt_subword_alpha

In [14]:
from onmt.bin.build_vocab import build_vocab_main
build_vocab_main(opts)

[2022-06-07 21:42:00,519 INFO] Parsed 2 corpora from -data.
[2022-06-07 21:42:00,520 INFO] Counter vocab from 10000 samples.
[2022-06-07 21:42:00,520 INFO] Build vocab on 10000 transformed examples/corpus.
[2022-06-07 21:42:00,525 INFO] corpus's transforms: TransformPipe()
[2022-06-07 21:42:00,741 INFO] Counters src:24995
[2022-06-07 21:42:00,742 INFO] Counters tgt:35816
